# **Solutions to dlt workshop Homework**

## Install requirements

In [1]:
%%capture
!pip install dlt[lancedb]==0.5.1a0
!pip install sentence-transformers

## Part 1: Create a Notion -> LanceDB pipeline using dlt

### 1. Install requirements

To create a notion -> lancedb pipeline, we need to install:
1. dlt with lancedb extras
2. sentence-transformers: we need to use an embedding model to vectorize and store data inside LanceDB. For this we choose the open-source model "sentence-transformers/all-MiniLM-L6-v2".

In [2]:
%%capture
!pip install dlt[lancedb]==0.5.1a0
!pip install sentence-transformers

### 2. Create a dlt project with rest_api source and lancedb destination

We now create a dlt project using the command `dlt init <source> <destination>`.

This downloads all the modules required for the dlt source (rest api, in this case) into the local directory. See the side panel for the directory structure created.

What is the dlt rest api source?

It is a dlt source that allows you to connect to any REST API endpoint using a declarative configuration. You can:
- pass the endpoints that you want to connect to,
- define the relation between the endpoints
- define how you want to handle pagination and authentication

In [3]:
!yes | dlt init rest_api lancedb

Looking up the init scripts in https://github.com/dlt-hub/verified-sources.git...
Cloning and configuring a verified source rest_api (Generic API Source)
Do you want to proceed? [Y/n]: 
Verified source rest_api was added to your project!
* See the usage examples and code snippets to copy from rest_api_pipeline.py
* Add credentials for lancedb and other secrets in ./.dlt/secrets.toml
* requirements.txt was created. Install it with:
pip3 install -r requirements.txt
* Read https://dlthub.com/docs/walkthroughs/create-a-pipeline for more information


### 3. Add API credentials

In [4]:
import os
from google.colab import userdata

os.environ["SOURCES__REST_API__NOTION__API_KEY"] = "secret_q4QgENGc9JKXgGContzdV6Z6KMTwzLYXWX9VMFi1dlb"

os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL_PROVIDER"] = "sentence-transformers"
os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL"] = "all-MiniLM-L6-v2"

os.environ["DESTINATION__LANCEDB__CREDENTIALS__URI"] = ".lancedb"
# os.environ["DESTINATION__LANCEDB__CREDENTIALS__API_KEY"] = "api_key"
os.environ["DESTINATION__LANCEDB__CREDENTIALS__EMBEDDING_MODEL_PROVIDER_API_KEY"] = userdata.get("HF_TOKEN")

### 4. Write the pipeline code

In [5]:
import dlt
from rest_api import RESTAPIConfig, rest_api_source

from dlt.sources.helpers.rest_client.paginators import BasePaginator, JSONResponsePaginator
from dlt.sources.helpers.requests import Response, Request

from dlt.destinations.adapters import lancedb_adapter

## **Q1. Rows in LanceDB**

How many rows does the lancedb table "notion_pages__homework" have?

14
15
16
17

In [6]:
from datetime import datetime, timezone

class PostBodyPaginator(BasePaginator):
    def __init__(self):
        super().__init__()
        self.cursor = None

    def update_state(self, response: Response) -> None:
        # Assuming the API returns an empty list when no more data is available
        if not response.json():
            self._has_next_page = False
        else:
            self.cursor = response.json().get("next_cursor")
            if self.cursor is None:
                self._has_next_page = False

    def update_request(self, request: Request) -> None:
        if request.json is None:
            request.json = {}

        # Add the cursor to the request body
        request.json["start_cursor"] = self.cursor

@dlt.resource(name="homework")
def rest_api_notion_resource():
    notion_config: RESTAPIConfig = {
        "client": {
            "base_url": "https://api.notion.com/v1/",
            "auth": {
                "token": dlt.secrets["sources.rest_api.notion.api_key"]
            },
            "headers": {
                "Content-Type": "application/json",
                "Notion-Version": "2022-06-28"
            }
        },
        "resources": [
            {
                "name": "search",
                "endpoint": {
                    "path": "search",
                    "method": "POST",
                    "paginator": PostBodyPaginator(),
                    "json": {
                        "query": "Homework: Employee handbook",
                        "sort": {
                            "direction": "ascending",
                            "timestamp": "last_edited_time"
                        }
                    },
                    "data_selector": "results"
                }
            },
            {
                "name": "page_content",
                "endpoint": {
                    "path": "blocks/{page_id}/children",
                    "paginator": JSONResponsePaginator(),
                    "params": {
                        "page_id": {
                            "type": "resolve",
                            "resource": "search",
                            "field": "id"
                        }
                    }
                }
            }
        ]
    }

    yield from rest_api_source(notion_config, name="homework")


def extract_page_content(response):
    block_id = response["id"]
    last_edited_time = response["last_edited_time"]
    block_type = response.get("type", "Not paragraph")
    if block_type != "paragraph":
        content = ""
    else:
        try:
            content = response["paragraph"]["rich_text"][0]["plain_text"]
        except IndexError:
            content = ""
    return {
        "block_id": block_id,
        "block_type": block_type,
        "content": content,
        "last_edited_time": last_edited_time,
        "inserted_at_time": datetime.now(timezone.utc)
    }

@dlt.resource(
    name="homework",
    write_disposition="merge",
    primary_key="block_id",
    columns={"last_edited_time": {"dedup_sort": "desc"}}
)
def rest_api_notion_incremental(
    last_edited_time=dlt.sources.incremental("last_edited_time", initial_value="2024-06-26T08:16:00.000Z", primary_key=("block_id"))
):
    for block in rest_api_notion_resource.add_map(extract_page_content):
        if not(len(block["content"])):
            continue
        yield block


def load_notion() -> None:
    pipeline = dlt.pipeline(
        pipeline_name="company_policies",
        destination="lancedb",
        dataset_name="notion_pages",
        # full_refresh=True
    )

    load_info = pipeline.run(
        lancedb_adapter(
            rest_api_notion_incremental,
            embed="content"
        ),
        table_name="homework",
        write_disposition="merge"
    )
    print(load_info)

load_notion()

_dlt_loads
[{'name': 'load_id', 'data_type': 'text', 'nullable': False}, {'name': 'schema_name', 'data_type': 'text', 'nullable': True}, {'name': 'status', 'data_type': 'bigint', 'nullable': False}, {'name': 'inserted_at', 'data_type': 'timestamp', 'nullable': False}, {'name': 'schema_version_hash', 'data_type': 'text', 'nullable': True}]
_dlt_pipeline_state
[{'name': 'version', 'data_type': 'bigint', 'nullable': False}, {'name': 'engine_version', 'data_type': 'bigint', 'nullable': False}, {'name': 'pipeline_name', 'data_type': 'text', 'nullable': False}, {'name': 'state', 'data_type': 'text', 'nullable': False}, {'name': 'created_at', 'data_type': 'timestamp', 'nullable': False}, {'name': 'version_hash', 'data_type': 'text', 'nullable': True}, {'name': '_dlt_load_id', 'data_type': 'text', 'nullable': False}, {'name': '_dlt_id', 'data_type': 'text', 'nullable': False, 'unique': True}]
_dlt_version
[{'name': 'version', 'data_type': 'bigint', 'nullable': False}, {'name': 'engine_version'

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

UPLOAD
Pipeline company_policies load step completed in 25.93 seconds
1 load package(s) were loaded to destination LanceDB and into dataset notion_pages
The LanceDB destination used <dlt.destinations.impl.lancedb.configuration.LanceDBCredentials object at 0x78af15d165f0> location to store data
Load package 1720559617.4866245 is LOADED and contains no failed jobs


In [7]:
import lancedb

db = lancedb.connect(".lancedb")
dbtable = db.open_table("notion_pages___homework")

dbtable.to_pandas()

,id__,vector__,block_id,block_type,content,last_edited_time,inserted_at_time,_dlt_load_id,_dlt_id
0,c69f1ecf-7b02-5810-8286-3f42659ae9d4,"[-0.024265556, 0.04746074, -0.01179647, 0.0638...",a8196881-ae94-4767-8767-92fe1a327d24,paragraph,We owe our success to our employees. To show o...,2024-07-05 22:34:00+00:00,2024-07-09 21:13:41.686657+00:00,1720559617.4866245,qq0W0haGhbopow
1,f2c18ac0-50f5-5b72-a871-dc5a46780353,"[-0.04966156, 0.10853508, -0.009762607, -0.036...",31fcbf26-2ca5-468a-8af8-d7eb4c2db8c8,paragraph,We want to ensure that private information abo...,2024-07-05 22:38:00+00:00,2024-07-09 21:13:41.692944+00:00,1720559617.4866245,NykGoxazaz9P1A
2,4553193e-c655-54df-9a33-cfc570bf34d0,"[-0.06316319, 0.17331503, 0.025351755, -0.0191...",da7721fd-3d0f-4c04-bc5e-825ad60bed1c,paragraph,Employee health is important to us. We don’t d...,2024-07-05 22:52:00+00:00,2024-07-09 21:13:41.693265+00:00,1720559617.4866245,PbCaZ4zPJTNk6Q
3,791be1a1-6c67-530d-87ab-bd9912500ea5,"[-0.109743185, 0.10586075, 0.003290699, -0.021...",ff36dcf3-5faa-40b4-ad8e-92fdc952201e,paragraph,Our company is dedicated to maintaining a safe...,2024-07-05 22:52:00+00:00,2024-07-09 21:13:41.693593+00:00,1720559617.4866245,qwg28TpQIrggNg
4,a83497f4-922c-5d62-bab1-53804e93c811,"[0.05242333, -0.064576, 0.06586297, 0.01454380...",a1ff9697-4bb6-4f1e-b464-dda296dbd307,paragraph,If your job doesn’t require you to be present ...,2024-07-05 22:52:00+00:00,2024-07-09 21:13:41.693916+00:00,1720559617.4866245,BDxoU9Mrv8L4iQ
5,434b71e9-a11a-519d-a9fe-e3ade78d47d6,"[0.00052337867, -0.054883413, 0.043573413, -0....",e4ec9f4d-b687-4c28-a80d-985bfabcc2ba,paragraph,Remote working refers to working from a non-of...,2024-07-05 22:52:00+00:00,2024-07-09 21:13:41.694232+00:00,1720559617.4866245,FEjSeHJr0zNuuA
6,17816363-54b7-5ba7-b8d5-06d871a25414,"[0.03802633, -0.021509705, 0.04752782, 0.06470...",e6e550dc-b59e-4928-abd7-07eace948681,paragraph,There are some expenses that we will pay direc...,2024-07-05 22:52:00+00:00,2024-07-09 21:13:41.694604+00:00,1720559617.4866245,1dEMBY67jOEW+w
7,2a434cf9-09d9-5514-a88b-02977f2f953e,"[-0.05858811, -0.07540446, 0.033775203, 0.0096...",a269d0ca-ce14-481b-a5f4-9192d6840d6e,paragraph,Our company operates between 9 a.m. to 7 p.m. ...,2024-07-05 22:52:00+00:00,2024-07-09 21:13:41.694926+00:00,1720559617.4866245,b7o0LDChD5HizA
8,5f9384fa-7f98-5f52-a06e-05b05f42f69a,"[-0.013599302, 0.0047530197, 0.024835136, 0.01...",5b65f3e7-0a37-429a-818d-f99b53755ebd,paragraph,"In this section, we are going to be covering i...",2024-07-05 23:33:00+00:00,2024-07-09 21:13:41.695248+00:00,1720559617.4866245,E4U4pTHozS5mQQ
9,42af72f6-9db7-54a2-87b2-d466169078ff,"[0.032060888, 0.024244698, 0.008471344, 0.0317...",b27f7d80-f2f1-460e-aa0c-b8e770cf050a,paragraph,Our company observes the following holidays: N...,2024-07-05 22:52:00+00:00,2024-07-09 21:13:41.695578+00:00,1720559617.4866245,GrEJYxya3GBiTg


### **A1.**

17.

 ---

## **Q2. Running the Pipeline: Last edited time**

In the demo, we created an incremental dlt resource rest_api_notion_incremental that keeps track of last_edited_time. What value does it store after you've run your pipeline once? (Hint: you will be able to get this value by performing some aggregation function on the column last_edited_time of the table)

Timestamp('2024-07-05 22:34:00+0000', tz='UTC') (OR "2024-07-05T22:34:00.000Z")
Timestamp('2024-07-05 23:33:00+0000', tz='UTC') (OR "2024-07-05T23:33:00.000Z")
Timestamp('2024-07-05 23:52:00+0000', tz='UTC') (OR "2024-07-05T23:52:00.000Z")
Timestamp('2024-07-05 22:56:00+0000', tz='UTC') (OR "2024-07-05T22:56:00.000Z")

### **A2.**

2024-07-05 22:56:00+00:00

## **Q3. Ask the Assistant**

Find out with the help of the AI assistant: how many PTO days are the employees entitled to in a year?

20
25
30
35

1. Install Ollama into the notebook's local runtime

In [8]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


2. Start Ollama using `ollama serve`. This needs to run in the backgound - so we run it using `nohup` (to see the output log, open nohup.out).

In [9]:
!nohup ollama serve > nohup.out 2>&1 &

3. Pull the desired model. We're going to be using `llama1-uncensored` (takes about 1m to download)

In [10]:
%%capture
!ollama pull llama2-uncensored

In this next part we're going to be writing functions that accept user question, retrieve the relevant paragraphs from lancedb, and the pass the question and the retrieved pages as input into the ollama chat assistant

4. pip install ollama and import it

In [11]:
!pip install -qU ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:00:00


In [12]:
import ollama

In [13]:
def retrieve_context_from_lancedb(dbtable, question, top_k=2):

    query_results = dbtable.search(query=question).to_list()
    context = "\n".join([result["content"] for result in query_results[:top_k]])

    return context

In [14]:
def main():
  # Connect to the lancedb table
  db = lancedb.connect(".lancedb")
  dbtable = db.open_table("notion_pages___homework")

  # A system prompt telling ollama to accept input in the form of "Question: ... ; Context: ..."
  messages = [
      {"role": "system", "content": "You are a helpful assistant that helps users understand policies inside a company's employee handbook. The user will first ask you a question and then provide you relevant paragraphs from the handbook as context. Please answer the question based on the provided context. For any details missing in the paragraph, encourage the employee to contact the HR for that information. Please keep the responses conversational."}
  ]

  while True:
    # Accept user question
    question = input("You: ")

    # Retrieve the relevant paragraphs on the question
    context = retrieve_context_from_lancedb(dbtable,question,top_k=2)

    # Create a user prompt using the question and retrieved context
    messages.append(
        {"role": "user", "content": f"Question: '{question}'; Context:'{context}'"}
    )

    # Get the response from the LLM
    response = ollama.chat(
        model="llama2-uncensored",
        messages=messages
    )
    response_content = response['message']['content']
    print(f"Assistant: {response_content}")

    # Add the response into the context window
    messages.append(
        {"role": "assistant", "content":response_content}
    )

In [ ]:
main()

You: PTO days
Assistant: Great! Here is my response based on the provided context:
"Thank you for your question. To answer it, I need to know what kind of information you would like about PTO days. Do you have any specific questions or concerns? If not, please let me know and I will try to help you out."
You: How many days for PTO?
Assistant: I understand your question and based on the context provided by our HRIS, I can confirm that employees receive 30 days of Paid Time Off (PTO) per year. This accrual begins the day you join our company and you receive 2.5 days per month. You can use your PTO at any time after your first week with us and you can use time off you haven’t accrued yet. However, we encourage you to use your PTO throughout the year. If you leave our company, we may compensate accrued PTO with your final paycheck according to local law. When the law doesn't have provisions, we will compensate accrued leave to employees who were not terminated for cause."


### **A3.**

30.